Ideas:
- For drawing one space: instead of up, down, left, right, combine into one, like "move(self, direction)" -- 
 And maybe use *args for directions (*directions), so can specify up to two directions??
 Otherwise, just use (self, h_dir='right', v_dir='down')
 then can use things like: 
  x = self.pos[0], y = self.pos[1]
  if 'left' in directions (or h_dir): x = x - 1, pos if 'up' in directions (or v_dir): y = y-1 (etc...)
  pos = [x, y]
  self.print(pos)

- Prob - turn doc with classes into module, which will be imported into main
  Create new doc to draw shapes, which imports the scribe module.
  Should (eventually) be able to call different functions from this doc, like "print_shape(shape, size)" directly from terminal, using terminal arguments (ex: $python drawstuff.py --shape square --size 5).
  New doc should be able to...
  - list shapes available (will need to create dictionary of all shapes available)
  - create_shape(shape)
  - control "shape size" (which will scale the shape depending on the number entered)
  - have other useful pre-set functions? Or perhaps these should live elsewhere (in main scribe module, or in another doc...)
  - ideally, eventually, have a help doc (list commands, args)

- Errors to catch:
  - TypeError (ex, if person puts string in for width or height)


In [33]:
import os
import time
from termcolor import colored

# This is the Canvas class. It defines some height and width, and a
# matrix of characters to keep track of where the TerminalScribes are moving


class Canvas:
    def __init__(self, width, height):
        self._x = width
        self._y = height
        # This is a grid that contains data about where the
        # TerminalScribes have visited
        self._canvas = [[' ' for y in range(self._y)] for x in range(self._x)]

    # Returns True if the given point is outside the boundaries of the Canvas
    def hitsWall(self, point):
        return point[0] < 0 or point[0] >= self._x or point[1] < 0 or point[1] >= self._y

    # Set the given position to the provided character on the canvas
    def setPos(self, pos, char):
        self._canvas[pos[0]][pos[1]] = char

    # Clear the terminal (used to create animation)
    def clear(self):
        os.system('cls' if os.name == 'nt' else 'clear')

    # Clear the terminal and then print each line in the canvas
    def print(self):
        self.clear()
        for y in range(self._y):
            print(' '.join([col[y] for col in self._canvas]))


class TerminalScribe:
    def __init__(self, canvas):
        self.canvas = canvas
        self.trail = '.'
        self.mark = '*'
        self.framerate = 0.2
        self.pos = [0, 0]

    def draw(self, pos):
        self.move_marks(pos)
        self.print_marks()

    # Move marker AND create trail in old position (but don't print)
    def move_marks(self, pos):
        # Set the old position to the "trail" symbol
        self.canvas.setPos(self.pos, self.trail)
        self.set_mark(pos)
    
    # Set the position of the mark
    def set_mark(self, pos):
        # Update position
        self.pos = pos
        # Set the new position to the "mark" symbol
        self.canvas.setPos(self.pos, colored(self.mark, 'red'))

    def print_marks(self):
        # Print everything to the screen
        self.canvas.print()
        # Sleep for a little bit to create the animation
        time.sleep(self.framerate)
    
    #set the current position to the "trail" symbol
    def end_line(self):
        self.canvas.setPos(self.pos, self.trail)
        self.print_marks()
    
    # Print a marker at the current self.pos
    def start_new_line(self):
        self.set_mark(self.pos)
        self.print_marks()
    
    # Jump mark position to new position relative to current, indicated by direction and number of spaces to move for each
    def jump(self, up=0, down=0, left=0, right=0, new=False):
        if new == False:
            self.end_line()
        x = (self.pos[0] + right) - left
        y = (self.pos[1] - up) + down

        # Update position
        self.pos = [x, y]
        

    # Animation:

    def up(self):
        pos = [self.pos[0], self.pos[1]-1]
        if not self.canvas.hitsWall(pos):
            self.draw(pos)

    def down(self):
        pos = [self.pos[0], self.pos[1]+1]
        if not self.canvas.hitsWall(pos):
            self.draw(pos)

    def right(self):
        pos = [self.pos[0]+1, self.pos[1]]
        if not self.canvas.hitsWall(pos):
            self.draw(pos)

    def left(self):
        pos = [self.pos[0]-1, self.pos[1]]
        if not self.canvas.hitsWall(pos):
            self.draw(pos)
    
    # Works
    def diag_up(self, dir_horizontal='right'):
        if dir_horizontal == 'right':
            x = self.pos[0] + 1
        else:
            x = self.pos[0] - 1

        pos = [x, self.pos[1] - 1]
        if not self.canvas.hitsWall(pos):
            self.draw(pos)
  
    # Works
    def diag_down(self, dir_horizontal='right'):
        y = self.pos[1] + 1
        if dir_horizontal == 'right':
            x = self.pos[0] + 1
        else:
            x = self.pos[0] - 1

        pos = [x, y]
        if not self.canvas.hitsWall(pos):
            self.draw(pos)



    # Draw Lines and Shapes:

    # works
    def draw_line(self, length=2, direction='right', new=False, end=False):
        for _ in range(length):
            if direction == 'up':
                self.up()
            elif direction == 'down':
                self.down()
            elif direction == 'right':
                self.right()
            elif direction == 'left':
                self.left()

    def draw_diag_up(self, length=2, direction='right'):
        for _ in range(int(length)):
            self.diag_up(direction)

    def draw_diag_down(self, length=2, direction='right'):
        for _ in range(int(length)):
            self.diag_down(direction)

    # works
    def draw_rectangle(self, width, height):
        w = width
        h = height - 1
        self.draw_line(w, 'right')
        self.draw_line(h, 'down')
        self.draw_line(w, 'left')
        self.draw_line(h, 'up')
        
        # to maybe add... ability to choose which corner to start drawing from? Plus draw direction (clockwise, counter-clockwise)...

    # ...draw arc = 1/4 of a circle?
    def draw_arc(self, radius):
        pass
        # ?? diagupright * r, right * r-1, diagdownright * r, down * r-2, diagdownleft * r, left * r-1, diagupleft * r, up * r-2


#class ScribeShape(TerminalScribe): --> ??
    #draw shape(shape, h, w, position)

In [34]:
# Create a new Canvas instance
canvas = Canvas(10, 10)

# Create a new scribe and give it the Canvas object
s = TerminalScribe(canvas)

# Draw a 5x5 box, with checkmark
def make_checked_box(scribe, pos):
    # create square
    start_pos = pos
    scribe.start_new_line()
    scribe.draw_rectangle(5,5)
    
    # create checkmark
    start_pos
    scribe.jump(down=1, right=1)
    scribe.start_new_line()
    scribe.draw_diag_down(2)
    scribe.draw_diag_up(5)
    scribe.end_line()

start_pos = s.jump(down=3, right=1, new=True)
make_checked_box(s, start_pos)

                   
                   
                   
  *                
                   
                   
                   
                   
                   
                   
                   
                   
                   
  . *              
                   
                   
                   
                   
                   
                   
                   
                   
                   
  . . *            
                   
                   
                   
                   
                   
                   
                   
                   
                   
  . . . *          
                   
                   
                   
                   
                   
                   
                   
                   
                   
  . . . . *        
                   
                   
                   
                   
                   
                   


Bad pipe message: %s [b"\x8d\xda\xcd\xcc\x19C\x11\xaa\xa1\xd7\xa1%\xec\xe3,\xaasB \xf7\x03\xaf\x9fb\x96\xa0\xd2\xb6\xa3K E'\xa4n\xb8\xda\\\xdeG\x87u\xc8s\x80\x10\x01\xf2\xf2A>\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00", b'\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b"k\x9c\x1a\x1cW}\xf7\x10\xed\xc4\xe1\x1d\xbe\xc3\xe4\xa4\xb7h\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x

In [118]:
canvas = Canvas(10, 10)
s = TerminalScribe(canvas)



def write_letters(scribe, *letters):
    size = 1
    scale = size * 3
    scribe.jump()
    
    x = scribe.pos[0]
    y = scribe.pos[1]
    
    x_offset = scribe.pos[0] - scale
    y_offset = scribe.pos[1] + scale
    start = [x, y]
    # 'A'
    
    # if drawing letter would run off of canvas at top or right, offset letter start 
    if scribe.canvas.hitsWall([x + scale, y - scale]):
        if scribe.canvas.hitsWall([x + scale, y]):
            scribe.pos[0] = scribe.pos[0] - scale
        if scribe.canvas.hitsWall([x + scale, y]):
            scribe.pos[1] = scribe.pos[1] + scale
    
    #start = [x_offset, y_offset]
    #    scribe.place_mark([x, y + scale])

    scribe.draw_diag_up(scale)
    scribe.draw_diag_down(scale)
    scribe.jump(left=5, up=1)
    
    #scribe.canvas.setPos(scribe.pos, scribe.trail)
    #scribe.set_mark([start[0] + size, start[1] - size])
    
 

write_letters(s)


                                       
                                       
                                       
*                                      
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
  *                                    
.                                      
                                       


In [17]:
width = 3
height = 4
# This is a grid that contains data about where the
# TerminalScribes have visited
canvas = [[f'{y}' for y in range(height)] for x in range(width)]
print(canvas)

for y in range(height):
    print([item[y] for item in canvas])
#    print(' '.join([col[y] for col in canvas]))

canvas_alt = [['a1', 'a2', 'a3', 'a4'], ['b1', 'b2', 'b3', 'b4'], ['c1', 'c2', 'c3', 'c4']]
print(canvas_alt)
# [col[0] for col in canvas]

for y in range(height):
    print([item[y] for item in canvas_alt])
    
for y in range(height):
    print(' '.join([col[y] for col in canvas_alt]))


[['0', '1', '2', '3'], ['0', '1', '2', '3'], ['0', '1', '2', '3']]
['0', '0', '0']
['1', '1', '1']
['2', '2', '2']
['3', '3', '3']
[['a1', 'a2', 'a3', 'a4'], ['b1', 'b2', 'b3', 'b4'], ['c1', 'c2', 'c3', 'c4']]
['a1', 'b1', 'c1']
['a2', 'b2', 'c2']
['a3', 'b3', 'c3']
['a4', 'b4', 'c4']
a1 b1 c1
a2 b2 c2
a3 b3 c3
a4 b4 c4


In [ ]:

def draw_line_up(self, height):
        for _ in range(height):
            self.up()

    def draw_line_down(self, height):
        for _ in range(height):
            self.down()

    def draw_line_right(self, width):
        for _ in range(width):
            self.right()

    def draw_line_left(self, width):
        for _ in range(width):
            self.left()


In [ ]:
# square
#for loop, for i in range(width of square), run scribe.right()
#for loop, for i in range(height of square), run scribe.down()

width = 5
height = 7
for _ in range(width):
    scribe.right()

for _ in range(height):
    scribe.down()

for _ in range(width):
    scribe.left()

for _ in range(height):
    scribe.up()

In [ ]:
def jump(self, direction, spaces=2):

        if direction == 'left':
            x = self.pos[0] - spaces
        elif direction == 'right':
            x = self.pos[0] + spaces
        elif direction == 'up':
            y = self.pos[1] - direction['up']
        elif direction == 'down':
            y = self.pos[1] + spaces

        self.move_mark(self, [x, y])

    # Jump mark from current to 2 or more spaces diagonally
    def jump_diag(self, horiz, vert, spaces=2):
        if horiz == 'left':
            x = self.pos[0] - spaces
        elif horiz == 'right':
            x = self.pos[0] + spaces

        if vert == 'up':
            y = self.pos[1] - spaces
        elif vert == 'down':
            y = self.pos[1] + spaces

        self.move_mark(self, [x, y])






 